In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [3]:
#Data Generation

In [4]:
from proj1_helpers import *

labels, features, indices = load_csv_data('train.csv', sub_sample=False)
# b(background) -> -1 and s(signal) -> 1

In [6]:
from implementations import *


[ 2.74399774e-04 -8.81118331e-03 -1.66993563e-03 -1.35826773e-03
  2.18409937e-04  6.28062814e-04 -1.42250015e-03  1.65909683e-03
 -8.17620432e-05  2.63351851e-03 -1.54787003e-03  3.88486383e-03
  6.19900948e-04  6.19001938e-03 -4.73878671e-05 -2.65346204e-04
  1.32206469e-03 -6.07759931e-05  1.99297029e-04  4.20603044e-03
  8.62878297e-05 -7.33832031e-04 -1.28725112e-03  1.09973808e-03
 -4.98935375e-04 -4.52710376e-04 -1.02293462e-04  1.72273626e-04
  1.29716901e-05 -4.87852712e-03]


(array([ 2.74399774e-04, -8.81118331e-03, -1.66993563e-03, -1.35826773e-03,
         2.18409937e-04,  6.28062814e-04, -1.42250015e-03,  1.65909683e-03,
        -8.17620432e-05,  2.63351851e-03, -1.54787003e-03,  3.88486383e-03,
         6.19900948e-04,  6.19001938e-03, -4.73878671e-05, -2.65346204e-04,
         1.32206469e-03, -6.07759931e-05,  1.99297029e-04,  4.20603044e-03,
         8.62878297e-05, -7.33832031e-04, -1.28725112e-03,  1.09973808e-03,
        -4.98935375e-04, -4.52710376e-04, -1.02293462e-04,  1.72273626e-04,
         1.29716901e-05, -4.87852712e-03]), 0.3567365083529938)